<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-data-from-tsv-source" data-toc-modified-id="Read-data-from-tsv-source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read data from tsv source</a></span></li><li><span><a href="#Connect-to-database" data-toc-modified-id="Connect-to-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Connect to database</a></span></li><li><span><a href="#Save-subreddit-category-info" data-toc-modified-id="Save-subreddit-category-info-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save subreddit category info</a></span></li><li><span><a href="#Cleaning-function" data-toc-modified-id="Cleaning-function-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cleaning function</a></span></li><li><span><a href="#Create-new-column-in-dataframe" data-toc-modified-id="Create-new-column-in-dataframe-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create new column in dataframe</a></span></li><li><span><a href="#Load-spaCy" data-toc-modified-id="Load-spaCy-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Load spaCy</a></span></li><li><span><a href="#Iterate-over-all-rows-and-perform-NLP" data-toc-modified-id="Iterate-over-all-rows-and-perform-NLP-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Iterate over all rows and perform NLP</a></span></li><li><span><a href="#Check-results" data-toc-modified-id="Check-results-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Check results</a></span></li><li><span><a href="#Save-to-database" data-toc-modified-id="Save-to-database-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Save to database</a></span></li></ul></div>

# Preparations

In [ ]:
import pandas as pd

## Read data from tsv source

In [ ]:
df = pd.read_csv("rspct.tsv", sep='\t')

Check if data is correct

In [ ]:
df

## Connect to database

In [ ]:
import sqlite3
con = sqlite3.connect('selfposts.db')

## Save subreddit category info

In [ ]:
pd.read_csv("subreddit_info.csv").to_sql("categories", con)

# Cleaning process

## Cleaning function

In [ ]:
import re
def clean(s):
    s = s.replace(r'<lb>', "\n")
    s = s.replace(r'<tab>', "\i")
    s = re.sub(r'<br */*>', "\n", s)
    s = s.replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    s = s.replace("&amp;", "&")
    # markdown urls
    s = re.sub(r'\(https*://[^\)]*\)', "", s)
    # normal urls
    s = re.sub(r'https*://[^\s]*', "", s)
    s = re.sub(r'_+', ' ', s)
    s = re.sub(r'"+', '"', s)
    return str(s)

## Create new column in dataframe

In [ ]:
df["selftext_clean"] = ''

# Iterate and clean

In [ ]:
for i, row in df.iterrows():
    df.at[i, "selftext_clean"] = clean(row.selftext)

Check results

In [ ]:
df.head()

# NLP

## Load spaCy

In [ ]:
import spacy
nlp = spacy.load('en')

## Iterate over all rows and perform NLP

In [ ]:
for i, row in df.iterrows():
    if i % 1000 == 0:
        print(i)
    if(row["selftext_clean"] and len(str(row["selftext_clean"])) < 1000000):
        doc = nlp(str(row["selftext_clean"]))
        adjectives = []
        nouns = []
        verbs = []
        lemmas = []

        for token in doc:
            lemmas.append(token.lemma_)
            if token.pos_ == "ADJ":
                adjectives.append(token.lemma_)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN":
                nouns.append(token.lemma_)
            if token.pos_ == "VERB":
                verbs.append(token.lemma_)
                
        df.at[i, "selftext_lemma"] = " ".join(lemmas)                
        df.at[i, "selftext_nouns"] = " ".join(nouns)
        df.at[i, "selftext_adjectives"] = " ".join(adjectives)
        df.at[i, "selftext_verbs"] = " ".join(verbs)
        df.at[i, "selftext_nav"] = " ".join(nouns+adjectives+verbs)
        df.at[i, "no_tokens"] = len(lemmas)

## Check results

In [ ]:
df.head()

## Save to database

In [ ]:
df.to_sql('posts_nlp', con)